Skin cancer detection

In [1]:
from keras_preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os, shutil
import pandas as pd

import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_digits, load_sample_images

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras import datasets, layers, models

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#%load_ext autoreload
#%autoreload 2

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (132 images), and reshape them
test_generator = ImageDataGenerator().flow_from_directory(
        test_data_dir, batch_size=118)

# Get all the data in the directory data/train (790 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
        train_data_dir, batch_size=2239)

# Get all the data in the directory data/validation (132 images), and reshape them
#val_generator = ImageDataGenerator().flow_from_directory(
#        val_data_dir, classes=['test'])

# Create the datasets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
#val_images, val_labels = next(val_generator)

Found 118 images belonging to 9 classes.
Found 2239 images belonging to 9 classes.


In [ ]:
# Preview an image
array_to_img(train_images[0])

In [ ]:
# Preview an image
array_to_img(test_images[0])

In [ ]:
train_labels.shape

In [ ]:
test_labels.shape

In [ ]:
test_images.shape

In [ ]:
train_images.shape

In [ ]:
train_labels

In [ ]:
type(train_labels)

In [ ]:
type(train_images)

In [ ]:
dataframe=pd.DataFrame(train_labels, columns=['actinic_keratosis', 'basal_cell_carcinoma',\
            'dermatofibroma', 'melanoma', 'nevus', 'pigmented_benign_keratosis',\
            'seborrheic_keratosis', 'squamous_cell_carcinoma', 'vascular_lesion'])

In [ ]:
dataframe

In [ ]:
sums=dataframe.sum(axis=0)

In [ ]:
sums=pd.DataFrame(sums)

In [ ]:
sums

In [ ]:
sums['%'] = ((sums[0] / 2239*100).round(2).astype(str) + '%')

In [ ]:
sums

In [ ]:
train_generator.class_indices 

In [ ]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
print(train_img.shape)
print(test_img.shape)

In [ ]:
train_y = np.reshape(train_labels[:,0], (2239,1))
test_y = np.reshape(test_labels[:,0], (118,1))

In [ ]:
X=train_img
y=train_y

In [ ]:
model1 = Sequential()

In [ ]:
model1.add(layers.Flatten())
model1.add(Dense(24, activation='relu', input_shape=(196608,)))
model1.add(Dense(12, activation='relu'))
model1.add(Dense(9, activation='softmax'))

In [ ]:
model1.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#model1.fit(X, y, epochs=50, batch_size=10)

The model did very well almost right away, 95% accuracy already at the fourth epoch.
Let us explore a bit more loss and accuracy using also a validation split.

In [ ]:
results1 = model1.fit(X, y, epochs=20, batch_size=10, validation_split=0.2)

In [ ]:
def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
visualize_training_results(results1)

I just want to see out of curiosity what is the performance, with the same exact parameters, if I were to import the images in a smaller size.

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (132 images), and reshape them
test_generator64 = ImageDataGenerator().flow_from_directory(
        test_data_dir, target_size=(64, 64), batch_size=10)

# Get all the data in the directory data/train (790 images), and reshape them
train_generator64 = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(64, 64), batch_size=20)


# Create the datasets
train_images64, train_labels64 = next(train_generator64)
test_images64, test_labels64 = next(test_generator64)

In [ ]:
train_img64 = train_images64.reshape(train_images64.shape[0], -1)
test_img64 = test_images64.reshape(test_images64.shape[0], -1)
print(train_img64.shape)
print(test_img64.shape)

In [ ]:
train_y64 = np.reshape(train_labels64[:,0], (20,1))
test_y64 = np.reshape(test_labels64[:,0], (10,1))

In [ ]:
X64=train_img64
y64=train_y64

In [ ]:
model2 = Sequential()

In [ ]:
model2.add(Dense(24, activation='relu', input_shape=(12288,)))
model2.add(Dense(12, activation='relu'))
model2.add(Dense(9, activation='softmax'))

In [ ]:
model2.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#model2.fit(X64, y64, epochs=50, batch_size=10)

In [ ]:
results2 = model2.fit(X64, y64, epochs=5, batch_size=10, validation_split=0.2)

In [ ]:
visualize_training_results(results2)

In [ ]:
history2 = results2.history
history2

Look at lab on https://github.com/learn-co-curriculum/dsc-image-classification-with-mlps-lab/tree/solution
might need to do some one hot encoding.
Also, just like it shows there, as a naive model maybe decrease the number of layers and keep epochs at like 5.
If results are the same, try to understand why the loss still showes such high peaks.And why the accuracy drops like that.
Might also be worth doing CNN from the start with 2D layers instead of this basic NN.

### Maybe another possible naive:

In [ ]:
model3 = Sequential()
model3.add(Dense(12, activation='tanh', input_shape=(12288,)))
model3.add(Dense(9, activation='softmax'))

In [ ]:
model3.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
results3 = model3.fit(X64, y64, epochs=5, batch_size=10, validation_split=0.2)

In [ ]:
visualize_training_results(results3)

In [ ]:
# changing batch size to 1
results4 = model3.fit(X64, y64, epochs=5, batch_size=1, validation_split=0.2)

In [ ]:
visualize_training_results(results4)

In [ ]:
# increasing batch size to 20
results5 = model3.fit(X64, y64, epochs=5, batch_size=20, validation_split=0.2)

In [ ]:
visualize_training_results(results5)

One more attempt at a naive model:
<br>since these ones keep performing even too well.
<br>I decrease again the size of the images, batch size and also number of neurons.

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (132 images), and reshape them
test_generator32= ImageDataGenerator().flow_from_directory(
        test_data_dir, target_size=(32, 32), batch_size=5)

# Get all the data in the directory data/train (790 images), and reshape them
train_generator32 = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(32, 32), batch_size=10)


# Create the datasets
train_images32, train_labels32 = next(train_generator32)
test_images32, test_labels32 = next(test_generator32)

In [ ]:
train_img32 = train_images32.reshape(train_images32.shape[0], -1)
test_img32 = test_images32.reshape(test_images32.shape[0], -1)
print(train_img32.shape)
print(test_img32.shape)

In [ ]:
train_y32 = np.reshape(train_labels32[:,0], (10,1))
test_y32 = np.reshape(test_labels32[:,0], (5,1))

In [ ]:
X32=train_img32
y32=train_y32

In [ ]:
model6 = Sequential()

In [ ]:
model6.add(Dense(11, activation='relu', input_shape=(3072,)))
model6.add(Dense(10, activation='relu'))
model6.add(Dense(9, activation='softmax'))

In [ ]:
model6.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
results6 = model6.fit(X32, y32, epochs=5, batch_size=5, validation_split=0.2)

In [ ]:
visualize_training_results(results6)

Even better, this does not make any sense.

Let us forget about all of this and try with the 2D actual convolutional neural network for images. The perfmormance there might be different.

We can also do something else which is group together the types of skin abnormalities that are cancerous and the ones that are not, reducing the classes to 2 and making this a binary classification problem.
I will do that later on. For now let's see what happens with a convolutional neural network, which is what I should be doing anyway.

## CNN

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (118 images), and reshape them
test_gen= ImageDataGenerator().flow_from_directory(
        test_data_dir, target_size=(32, 32), batch_size=118)

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(32, 32), batch_size=2239)

# Create the datasets
train_img, train_lab = next(train_gen)
test_img, test_lab = next(test_gen)

In [ ]:
# Explore your dataset again
m_train = train_img.shape[0]
num_px = train_img.shape[1]
m_test = test_img.shape[0]
#m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
#print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_img.shape))
print ("train_labels shape: " + str(train_lab.shape))
print ("test_images shape: " + str(test_img.shape))
print ("test_labels shape: " + str(test_lab.shape))
#print ("val_images shape: " + str(val_images.shape))
#print ("val_labels shape: " + str(val_labels.shape))

In [ ]:
#train_img = train_img.reshape(train_img.shape[0], -1)
#test_img = test_img.reshape(test_img.shape[0], -1)
#val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
#print(val_img.shape)

In [ ]:
train_y = np.reshape(train_lab[:,0], (2239,1))
test_y = np.reshape(test_lab[:,0], (118,1))

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
#                         input_shape=(3072)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(10, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(9, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history = model.fit(train_img,
                    train_y,
                    epochs=30,
                    batch_size=10, validation_split=0.2)

In [ ]:
history.history

In [ ]:
def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_acc'])
    plt.plot(history['acc'])
    plt.legend(['val_accuracy', 'accuracy'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
visualize_training_results(history)

Trying with an even more simple model:

In [ ]:
model2 = models.Sequential()
model2.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
model2.add(layers.MaxPooling2D((2, 2)))

#model.add(layers.Conv2D(10, (4, 4), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))

#model.add(layers.Conv2D(32, (3, 3), activation='relu'))
#model.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Flatten())
model2.add(layers.Dense(32, activation='relu'))
model2.add(layers.Dense(9, activation='softmax'))

model2.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

And doing only 10 epochs:

In [ ]:
history2 = model2.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=10, validation_split=0.2)

In [ ]:
visualize_training_results(history2)

Let me try with a larger batch size:

In [ ]:
history3 = model2.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=50, validation_split=0.2)

In [ ]:
visualize_training_results(history3)

Last attempt: all the previous parameters but making the last dense layer smaller.

In [ ]:
model3 = models.Sequential()
model3.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
model3.add(layers.MaxPooling2D((2, 2)))

model3.add(layers.Flatten())
model3.add(layers.Dense(10, activation='relu'))
model3.add(layers.Dense(9, activation='softmax'))

model3.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history4 = model3.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=50, validation_split=0.2)

In [ ]:
visualize_training_results(history4)

Even less Neurons:

In [ ]:
model4 = models.Sequential()
model4.add(layers.Conv2D(5, (3, 3), activation='relu',
                        input_shape=(32 ,32,  3)))
model4.add(layers.MaxPooling2D((2, 2)))

model4.add(layers.Flatten())
model4.add(layers.Dense(5, activation='relu'))
model4.add(layers.Dense(9, activation='softmax'))

model4.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history5 = model4.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=50, validation_split=0.2)

In [ ]:
visualize_training_results(history5)

No that doesn't change. Ok.
Let's move on and try to improve our model.

#### No sorry one more thing. Even less resolution images.

In [ ]:
# Directory path
train_data_dir = 'dataskin/Train'
test_data_dir = 'dataskin/Test'
#val_data_dir = 'dataset/validation_set'

# Get all the data in the directory data/validation (118 images), and reshape them
test_gen= ImageDataGenerator().flow_from_directory(
        test_data_dir, target_size=(16, 16), batch_size=118)

# Get all the data in the directory data/train (2239 images), and reshape them
train_gen = ImageDataGenerator().flow_from_directory(
        train_data_dir, target_size=(16, 16), batch_size=2239)

# Create the datasets
train_img, train_lab = next(train_gen)
test_img, test_lab = next(test_gen)

In [ ]:
# Explore your dataset again
m_train = train_img.shape[0]
num_px = train_img.shape[1]
m_test = test_img.shape[0]
#m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
#print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_img.shape))
print ("train_labels shape: " + str(train_lab.shape))
print ("test_images shape: " + str(test_img.shape))
print ("test_labels shape: " + str(test_lab.shape))

In [ ]:
train_y = np.reshape(train_lab[:,0], (2239,1))
test_y = np.reshape(test_lab[:,0], (118,1))

In [ ]:
model5 = models.Sequential()
model5.add(layers.Conv2D(10, (3, 3), activation='relu',
                        input_shape=(16 ,16,  3)))
model5.add(layers.MaxPooling2D((2, 2)))

model5.add(layers.Flatten())
model5.add(layers.Dense(10, activation='relu'))
model5.add(layers.Dense(9, activation='softmax'))

model5.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [ ]:
history6 = model5.fit(train_img,
                    train_y,
                    epochs=10,
                    batch_size=50, validation_split=0.2)

In [ ]:
visualize_training_results(history5)

### Starting to improve our model: